In [15]:
from curvaturemodel import curvature as curv 
import pandas as pd
import os
import sys
sys.path.append('/Users/cmagri1/OneDrive - Johns Hopkins/git')
from EncodingModel_cm296 import utils as emutils


In [16]:
#where to get the images from
PathToImgs = 'data-object2vec/stimuli'

#where to load word2sense
pathtoW2S = '../code-00-preprocessdataset/'
Wrd2Sense = pd.read_csv(pathtoW2S + "ThingsWrd2Sns.txt", sep=",")
WordInf = pd.read_csv(pathtoW2S + "KeptTHINGSInfo.txt", sep=",")['Word'].to_numpy()
# .Word.to_numpy()
# WordInf= ImgInfo.Word.to_numpy() ##to find the words that are images in THINGS dataset


pathtoTHINGS = '../../../../THINGSdataset/Main/images'
savepath = '../../../data-00/'
pretrainedModel = False

In [35]:
import numpy as np
from tqdm import tqdm
import torch
from PIL import Image
from torchvision.transforms import functional as tr
from torchvision import transforms as tt

def listdir(dir, path=True):
    files = os.listdir(dir)
    files = [f for f in files if f != '.DS_Store']
    files = sorted(files)
    if path:
        files = [os.path.join(dir, f) for f in files]
    return files

imagenet_mean = (0.485, 0.456, 0.406)
imagenet_std = (0.229, 0.224, 0.225)


def image_to_tensor(image, resolution=None,paddingval=None,padding_mode = 'constant', do_imagenet_norm=False, do_padding = True):
    if isinstance(image, str):
        image = Image.open(image).convert('LA')     
        if image.width != image.height:     # if not square image, crop the long side's edges to make it square
            r = min(image.width, image.height)
            image = tr.center_crop(image, (r, r))
        if do_padding:     # if not square image, crop the long side's edges to make it square
            image = tr.pad(image, padding=paddingval, padding_mode=padding_mode, fill=0)
            # image = tr.pad(input=data, mode='reflect', value=0)
        if resolution is not None:#f size is an int, smaller edge of the image will be matched to this number
            image = tr.resize(image, resolution)
    
        image = tr.to_tensor(image)
        if do_imagenet_norm:
            image = imagenet_norm(image)
    return image

def imagenet_norm(image):
    dims = len(image.shape)
    if dims < 4:
        image = [image]
    image = [tr.normalize(img, mean=imagenet_mean, std=imagenet_std) for img in image]
    image = torch.stack(image, dim=0)
    if dims < 4:
        image = image.squeeze(0)
    return image

def compute_features_curvature(model, conditionsPath, resolutionval = 227,WordInf =[],paddingval=0,padding_mode='constant'):
    #takes model and loads the features for that image, needs path to of files in directory

    conditions = listdir(conditionsPath)
    # print(len(conditions))

    if WordInf != np.array([]):
        new_conditions = []
        for c in conditions:
            w = c.split('/')[-1]
            if w in WordInf:
                new_conditions.append(c)
        conditions = new_conditions


    condition_features = {}
    for c in tqdm(conditions):
        c_name = c.split('/')[-1]
        
        stimuli = listdir(c)
        #resize according to resolution and square the image
        stimuli = [image_to_tensor(s, resolution=resolutionval,paddingval=paddingval,padding_mode = padding_mode) for s in stimuli]
        stimuli = torch.stack(stimuli)
       
        #average across the same category
        feats = model.forward(stimuli).numpy()
    
#         condition_features[c_name] = feats
    return feats

In [36]:

#Compute DNN features
def object_by_feature(PathToImgs,namemodel,keyword, savepath,WordList=[]):

    namefile = savepath+ keyword + '_' + namemodel 
        
    if os.path.isfile(namefile+".csv"): #if it's already saves
        
        print('loading file with '+ keyword+ ' stimuli features for ' + namemodel + ' model')
        features = pd.read_csv(namefile+".csv"  , sep=",", header=None, index_col=0)
        condition_features = {}
        for index, row in features.iterrows():
            condition_features[index] = row 
            
            
    else:
        
        print('Computing '+ keyword+ ' stimuli features for ' + namemodel + ' model')
        #Specify the model
        if namemodel is 'Curvature':
            model = curv.CurvatureModel()
        condition_features = compute_features_curvature(model, PathToImgs,WordInf = WordList)
#         features = 
        pd.DataFrame(condition_features).transpose().to_csv(namefile+".csv", index = True, header=False)
#         features.to_csv(namefile+".csv", index = True, header=False)
        np.save(namefile,condition_features)
#     layer_by_features[ilayer] = features; #create multistructure with all layers
    return condition_features

In [37]:
#Compute features of CNN
#---------Object2Vect-----------------
model = 'Curvature'
keyword = 'object2vec'
object2vec_features = object_by_feature(PathToImgs,model,keyword,savepath)
    

/Users/cmagri1/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  0%|          | 0/81 [00:00<?, ?it/s]

Computing object2vec stimuli features for Curvature model


RuntimeError: Given groups=1, weight of size 320 1 9 9, expected input[10, 2, 227, 227] to have 1 channels, but got 2 channels instead